# Advent of Code 2022: Day 16
https://adventofcode.com/2022/day/16


## Part 1
Release the most amount of pressure in 30 minutes

### Get the data into a list of strings

In [1]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n')
# Remove empty value at the bottom of the list.
data_list = data_list[:-1]

### Dijkstra algorithm to find the shortest path from each room to every other room

In [2]:
def dijkstra(data, start, goal):
  
  # Setup lists and dictionaries for
  # performing the shortest-path search.
  exploreQ = []
  exploreQ.append(start)

  prev = {}
  prev[start] = None

  dist = {}
  dist[start] = 0

  # Perform the search. Save the shortest
  # distance to the starting room. 
  while exploreQ:
    curr = exploreQ.pop(0)
    if curr == goal:
      break
    
    for neighbor in data[curr][1]:
      new_dist = dist[curr] + 1
      if (neighbor not in dist.keys()) or (new_dist < dist[neighbor]):
        dist[neighbor] = new_dist
        exploreQ.append(neighbor)
        prev[neighbor] = curr
  return dist

### Function to preprocess the data

In [3]:
def preprocess_data(data):
  
  # Setup dictionaries to return.
  output_dict = {}
  dists = {}
  valves = []

  # Go through each valve and parse
  # the text information into the
  # desired format, where the name
  # of the valve is the key and the
  # value is a list, where the first 
  # element is its flow value and the
  # second element is a list of valves 
  # that can be reached from this valve.
  for item in data:
    split = item.split('; ')
    valve = split[0].split(' ')[1]
    flow = int(split[0].split('=')[1])
    tunnels = [] 
    for v in split[1].split(' ')[4:]:
      tunnels.append(v.rstrip(','))

    valves.append(valve)
    output_dict[valve] = [flow, tunnels]

  # Go through every vavle, if it is
  # the starting valve 'AA' or if it 
  # has a positive flow value, and
  # calculate the shortest path to 
  # every other valve that has a 
  # positive flow value. 
  for v in valves:
    start = v
    if v == 'AA' or output_dict[v][0] > 0:
      dists[v] = {}
      for g in valves:
        if (g == 'AA' and g!=v) or (g != v and output_dict[g][0] >0):
          dists[v][g] = dijkstra(output_dict, v, g)[g]
        
  return dists, output_dict

In [4]:
dists, data_dict = preprocess_data(data_list)

### Function to setup the minutes array

In [5]:
def setup_minutes(dists, n_minutes):
  # Setup a list of empty lists
  # with a length equal to the 
  # number of minutes.
  minutes = []
  for i in range(n_minutes):
    minutes.append([])
  
  # For each minute, add a 2d-array
  # with rows equal to the number of 
  # positive valves and columns equal
  # to 2 to the power of the number
  # of positive valves. Make
  # all entries -inf.
  for i in range(len(minutes)):
    temp_list = []
    for j in range(len(dists)):
      inner_temp_list = []
      for l in range(2**(len(dists))):
        inner_temp_list.append(float('-inf'))
      temp_list.append(inner_temp_list)
    minutes[i] = (temp_list)
  
  # Starting at valve 'AA' go to 
  # each of the positive valves.
  # Update the minutes table with
  # 0 to represent that we are now
  # in that valve. Which minute to
  # update depends on the distance
  # from AA to a valve.
  valves = list(dists.keys())
  for j in range(len(valves)):
    if valves[j]!='AA':
      dist = dists['AA'][valves[j]]
      minutes[dist][j][1<<j] = 0
  return minutes

### Function to get the current pressure flow, which depends on how many valves are currently open, represented by l

In [6]:
def get_pressure(data_dict, l, valves):
  out_rate = 0
  for j in range(len(valves)):
    if (1<<j & l !=0):
      out_rate += data_dict[valves[j]][0] 
  return out_rate

### Function to fill the minutes table with the total pressure for every possible path.

In [7]:
def calculate_paths(dists, data_dict, n_minutes):
  # Setup the minutes array and get a 
  # list of the positive valves.
  minutes = setup_minutes(dists, n_minutes)
  valves = list(dists.keys())
  
  # Go through every minute, skipping
  # the first minute as nothing will
  # be turned on at this point. 
  for i in range(len(minutes)):
    if i == 0:
      continue
    else:
      
      # Go through every state (l) which corresponds
      # to the different permutations of valves being
      # turned on for every valve (j).
      for j in range(len(minutes[0])):
        for l in range(len(minutes[0][0])):
          
          # If the value at the same j and l for the previous
          # minute was -inf, this valve/state combination is
          # not possible to achieve, so skip it.
          if minutes[i-1][j][l] == float('-inf') and minutes[i][j][l] == float('-inf'):
            continue
          
          # Calculate the current pressure flow, calculate
          # the total and update the minutes list, unless
          # the value there is already equal or greater.
          temp_p = get_pressure(data_dict, l, valves)
          total_p = temp_p + minutes[i-1][j][l]
          if total_p > minutes[i][j][l]:
            minutes[i][j][l] =  total_p    
          
          # If the valves represented by
          # l are not on in the states
          # included in j, then we are
          # not currently at one of 
          # these valves, which means the
          # following part can be skipped. 
          if (1<<j & l) == 0:
            continue 
          
          # Go through the other positive
          # valves. 
          for ij in range(len(valves)):
            
            # If a valve is already
            # on, we do not need to visit 
            # it again.
            if 1<<ij & l !=0:
              continue

            # Otherwise, move from the current
            # valve to the other valves. Update
            # the minutes table depending on
            # how many minutes it takes to get
            # there, and what the current pressure
            # flow is. 
            if valves[j] != valves[ij]:
              dist = dists[valves[j]][valves[ij]]+1
              if i+dist <= (n_minutes-1):
                c = minutes[i][j][l] + temp_p*dist
                if c > minutes[i+dist][ij][l|(1<<ij)]:
                  minutes[i+dist][ij][l|(1<<ij)] = c  
  return minutes

In [8]:
minutes = calculate_paths(dists, data_dict, 30)
max([max(i) for i in minutes[29]])

2320

## Part 2
Release the most amount of pressure in 26 minutes, with two people moving through the tunnels

In [9]:
# Setup the maximum pressure counter,
# a list of the positive valves and
# extract the desired minute from
# the minutes array.
max_pressure = 0
valves = list(dists.keys())
twenty_six = minutes[25]

# Go through the chosed minute
# twice to simulate two people
# walking through the tunnel.
for i in range(len(twenty_six[0])):
  for j in range(len(twenty_six[0])):
    
    # If j is not a subset of i,
    # then there are valves turned
    # on for person i which are not
    # on for person j. These cases 
    # are not interesting as we want
    # to simulate two people turning
    # valves on, which means if a 
    # valve is turned on by either
    # person it should be considered
    # turned on for both. 
    if i & j !=j:
      continue
    
    # Set initial pressure values
    # for each person to -inf
    person1 = float('-inf')
    person2 = float('-inf')

    # Go through every positive valve
    # and update the maximum if the 
    # pressure at the current states
    # are greater than previous.
    for l in range(len(valves)):
      person1 = max(person1, twenty_six[l][j])
      person2 = max(person2, twenty_six[l][i&~j])
    max_pressure = max(max_pressure, person1+person2)
max_pressure

2967